Import packages

In [14]:
import sys
sys.path.append('../helpers/')

from pathlib import Path
import numpy as np
import os
from glob import glob
import natsort
import cv2
import csv
import random
from tqdm import tqdm

import matplotlib.pyplot as plt

# importing packages
from preprocessing import Preprocessing
from feature_extraction import FeatureExtraction

preprocessor = Preprocessing()
feature_extractor = FeatureExtraction()

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Read the images

In [5]:
# SAVE_PATH = Path("Processed-Heart-Detection")
# dcm_path = ROOT_PATH/str(patient_id) # joining paths
# dcm_path = dcm_path.with_suffix(".dcm") # adds suffix

ROOT_PATH = Path(Path(os.getcwd())/"../challenge1")
TRAIN_PATH = ROOT_PATH/"train"
VAL_PATH = ROOT_PATH/"val"

print(TRAIN_PATH)

c:\Users\abdal\Documents\Master\EMJMD MAIA\SEMESTER 3 - UdG\CAD\Melanoma-classification-using-machine-learning\notebooks\..\challenge1\train


In [6]:
train_nevus = sorted(glob(str(TRAIN_PATH/'nevus/*')))
train_others = sorted(glob(str(TRAIN_PATH/'others/*')))

val_nevus = sorted(glob(str(VAL_PATH/'nevus/*')))
val_others = sorted(glob(str(VAL_PATH/'others/*')))


In [7]:
# train_nevus

# TODO ASK:
# Do we normalize the images in pre-processing or after we extract all of the features?
# Should we remove the hair? Aggressivly? 
# Should we fill the borders?

In [8]:
def display_img(image, gray=False):
    if gray:
        plt.imshow(image, cmap='gray')
    else:
        # plt.imshow(image)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')


In [25]:
# Pre-processing and feature extraction
features_dir = r'../output/features/'

experiment              = 1
subsample               = True

nevus_train_filename    = f'{experiment}_nevus_train_features.csv'
nevus_val_filename      = f'{experiment}_nevus_val_features.csv'

others_train_filename   = f'{experiment}_others_train_features.csv'
others_val_filename     = f'{experiment}_others_val_features.csv'

random.seed(42)
random.shuffle(train_nevus)
random.shuffle(val_nevus)
random.shuffle(train_others)
random.shuffle(val_others)

filenames_list = [nevus_train_filename, nevus_val_filename, others_train_filename,  others_val_filename]
dir_list = [train_nevus, val_nevus, train_others, val_others]

labels = [0, 0, 1, 1]

for index, filename in enumerate(filenames_list):
    with open(os.path.join(features_dir, filename), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        for count, image_path in tqdm(enumerate(dir_list[index])):

            if subsample:
                if count == 999: # only 1k per class
                    break

            # reading the image 
            image = cv2.imread(image_path,cv2.IMREAD_COLOR)

            # 1. Copping black frame
            image_without_black_frame, _ = preprocessor.crop_frame(image)

            # 2. Resizing
            image_resized = preprocessor.resize_images(image_without_black_frame, preserve_ratio=True)

            # 3. Removing hair
            image_without_hair = preprocessor.extract_hair(image_resized)

            # 4. Displaying result
            # display_img(image_without_hair)
            
            # 5. Extracting features
            feature_vector = feature_extractor.fit(image_without_hair)

            # 6. Add label column
            feature_vector = np.append(feature_vector,labels[index])

            # print(feature_vector)
            writer.writerow(feature_vector)



0it [00:00, ?it/s]c:\Users\abdal\Documents\Master\EMJMD MAIA\SEMESTER 3 - UdG\CAD\Melanoma-classification-using-machine-learning\env\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\abdal\Documents\Master\EMJMD MAIA\SEMESTER 3 - UdG\CAD\Melanoma-classification-using-machine-learning\env\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
999it [01:34, 10.58it/s]
999it [01:24, 11.80it/s]
999it [02:18,  7.21it/s]
999it [02:18,  7.20it/s]


In [ ]:
# Image augmentation
# 1. Create dataset class (DL)
# 2. pass the augmentation as an argument (create class/file)